# Model Training
Model training can be a never ending step in machine learning, so knowing when to stop experimenting is inmportant. In this notebook we will look at how different models compare and how to finetune them. We will use a series of techniques to improve the accuracy of a model.

First we will load in our cleaned data from the last notebook.

In [98]:
import numpy as np
import pandas as pd

df = pd.read_parquet("solar_cleaned.parquet")

## Split Data
Now we will split the data into a testing set and a training set. Creating this test set is called hold-out validation and will be used as a sanity check for overfitting. Overfitting is when the model learns from the training set too well and preforms poorly on new data. There are other more in-depth validation techniques such as [cross-validation](https://en.wikipedia.org/wiki/Cross-validation_(statistics)), but we wont cover that in this demo.

In [99]:
middle = int(len(df) * 0.7)

train = df.loc[:middle].copy()
test = df.loc[middle:]

print("Training shape: " + str(train.shape))
print("Testing shape: " + str(test.shape))

Training shape: (30251, 39)
Testing shape: (12965, 39)


We will also load in the feature engineering pipeline from the previous notebook. Recall that this pipeline converts the raw data into normalized data that the model can use. We use this to seperate the inputs and outputs (x and y) of our training and testing data.

In [100]:
import cloudpickle as cp
#feature_pipeline = cp.load(open('feature_pipeline.sav', 'rb'))

#X_train = feature_pipeline.fit_transform(train)
X_train = train.drop(columns=['dni_efficiency', 'ghi_efficiency', 'dhi_efficiency', 'STATION','DATE',
                                            'latitude','longitude'])
y_train = train['dni_efficiency']

#X_test = feature_pipeline.fit_transform(test)
X_test = test.drop(columns=['dni_efficiency', 'ghi_efficiency', 'dhi_efficiency', 'STATION','DATE',
                                            'latitude','longitude'])
y_test = test['dni_efficiency']

## Training
We will train two different regression models: random forrest and xgboost. I removed the DHI and GHI efficiency outputs from the training set sense they were giving over 100% efficiency scores. This would cause the model to start predicting wrong values if they were used. Because we are only predicting efficiency, not training all of our outputs will help the model avoid more errors in the data.

### Random Forest
First we use sklearn's [Random Forest Regressor](https://en.wikipedia.org/wiki/Random_forest) for our first model. Random forest models work by creating many [decision trees](https://en.wikipedia.org/wiki/Decision_tree) and aggregating the average prediction of the combined trees. Each tree will split, for a desired amount of iterations, based on a random feature. There are many more parameters that can be tuned but to start we will just use the default.

In [101]:
from sklearn.ensemble import RandomForestRegressor
from sklearn import model_selection

rfr = RandomForestRegressor()
rfr.fit(X_train, y_train)

RandomForestRegressor()

### XGBoost
In parallel, we will train a gradient boosted tree model called [XGBoost](https://en.wikipedia.org/wiki/XGBoost). These trees differ from random forest trees in that each tree builds off the last compared to aggregating each tree. We will train it on default parameters also.

In [102]:
import xgboost as xgb

xg_reg = xgb.XGBRegressor()
xg_reg.fit(X_train,y_train)

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
             importance_type='gain', interaction_constraints='',
             learning_rate=0.300000012, max_delta_step=0, max_depth=6,
             min_child_weight=1, missing=nan, monotone_constraints='()',
             n_estimators=100, n_jobs=0, num_parallel_tree=1, random_state=0,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=1,
             tree_method='exact', validate_parameters=1, verbosity=None)

## Model Validation
Now we need to test to see how the models are preforming on data they have not seen. There are a number of metrics we can use, but to start we can use [R^2](https://en.wikipedia.org/wiki/Coefficient_of_determination). We use this metric to find the proportion of variance in our model which can show how well a model can predicts real values. The closer the R^2 value is to one, the more accurate our model is. 

In [103]:
from sklearn.metrics import r2_score, median_absolute_error

predictions = rfr.predict(X_test)
print("R^2 Scores\n------------\nRandom Forest: \t" + str(r2_score(y_test.values, predictions)))

predictions = xg_reg.predict(X_test)
print("XGBoost: \t" + str(r2_score(y_test.values, predictions)))

R^2 Scores
------------
Random Forest: 	0.7169902009334733
XGBoost: 	0.7151954636103839


Both scores are similer so we run another metric called the median absolute error. This metric is robust againts outliers which could help distingiush between the models. Smaller values equate to a better model preformance.

In [104]:
predictions = rfr.predict(X_test)
print("Median Absolute Error\n------------------------\nRandom Forest: \t" +
      str(median_absolute_error(y_test.values, predictions)))

predictions = xg_reg.predict(X_test)
print("XGBoost: \t" + str(median_absolute_error(y_test.values, predictions)))

Median Absolute Error
------------------------
Random Forest: 	0.09300238718967224
XGBoost: 	0.09418881670626102


## Hyperparameter Tuning
At the moment both models are preforming at an ok accuracy but it is not great. We can improve the models accuracy though a process called hyperparameter tuning. Here we will change the parameters for each model until a better accuracy is found. The method we will use to acomplish this is called random search. We will test a series of random parameters until we find the top preforming set.

First we define a sampling distribution for each hyperparameter that we want to change.


For the random forest model, we look at these main parameters: `n_estimators, max_depth, min_samples_split, min_samples_leaf, max_features`. Look at sklearns [documentation](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestRegressor.html) on this model for what each parameter means.

For the boosting model, we look at these main parameters: `n_estimators, max_depth, learning_rate, colsample_bynode, colsample_bytree, subsample`. Look at XGBoost's [documentation](https://xgboost.readthedocs.io/en/latest/python/python_api.html#module-xgboost.sklearn) on this model for what each parameter means.

In [105]:
from scipy.stats import randint as sp_randint
from scipy.stats import uniform

# both
n_estimators = sp_randint(10, 1000)
max_depth = sp_randint(1, 40)

# rf
min_samples_split = sp_randint(2, 5)
min_samples_leaf = sp_randint(1, 5)
max_features = sp_randint(1, 30)

# boost
learning_rate = uniform(loc=0, scale=0.05)
colsample_bynode = uniform(loc=0, scale=1)
colsample_bytree = uniform(loc=0, scale=1)
subsample = uniform(loc=0, scale=1)

Next we will create a function that will train each random model and output a Pandas DataFrame of its relevant values.

In [106]:
def record_model(model, model_type):
    model.fit(X_train, y_train)
    output = pd.DataFrame({
        'model_type': model_type,
        'model': [model],
        'r2_score': r2_score(y_test.values, model.predict(X_test))
    })
    return output

Finally we run though `itterations` amount of random tests.

In [107]:
itterations = 1

models = pd.DataFrame(columns=['model_type', 'model', 'r2_score'])
for i in range(itterations):
    model = RandomForestRegressor(n_estimators=n_estimators.rvs(),
                               max_depth=max_depth.rvs(),
                               min_samples_split=min_samples_split.rvs(),
                               min_samples_leaf=min_samples_leaf.rvs())
    models = models.append(record_model(model, 'random_forest'))
    
    model = xgb.XGBRegressor(n_estimators=n_estimators.rvs(),
                             max_depth=max_depth.rvs(),
                            learning_rate=learning_rate.rvs(),
                            colsample_bynode=colsample_bynode.rvs(),
                            colsample_bytree=colsample_bytree.rvs(),
                            subsample=subsample.rvs())
    models = models.append(record_model(model, 'boost'))

We can repeat this process with smaller random ranges to further tune the model, but for the sake of this demo we will take the top models with the highest R^2 value for each model type.

We plot the scores to see which sets of parameters preformed the best.

In [108]:
import altair as alt

models_2 = models.reset_index(drop=True)
models_2 = models_2.reset_index()

base = alt.Chart(models_2[['model_type', 'r2_score', 'index']])

bars = base.mark_bar(size=15).encode(
x=alt.X('index:Q', title="Model"),
y= alt.Y('r2_score:Q', scale=alt.Scale(zero=False)),
color='model_type:N')

text = bars.mark_text(
    baseline='top',
    dy=-15
).encode(
    text=alt.Text('r2_score:Q', format=',.3r')
)

(bars + text).properties(width=700)

alt.LayerChart(...)

Here we are extracting the best model for both model types and displaying the parameters that they used.

In [109]:
# top random forest
rfr = models[models['model_type']=='random_forest']
rfr = rfr.iloc[rfr['r2_score'].values.argmax()].values[1]
pd.DataFrame(rfr.get_params(), index=[0])[['n_estimators', 'max_depth',
                                           'min_samples_split', 'min_samples_leaf',
                                           'max_features']]

,n_estimators,max_depth,min_samples_split,min_samples_leaf,max_features
0,962,35,4,3,auto


In [110]:
# top boost
xg_reg = models[models['model_type']=='boost']
xg_reg = xg_reg.iloc[xg_reg['r2_score'].values.argmax()].values[1]

# save this model for later export
from copy import copy
saved_model = copy(xg_reg)

pd.DataFrame(xg_reg.get_params(), index=[0])[['n_estimators', 'max_depth',
                                              'learning_rate', 'colsample_bynode',
                                              'colsample_bytree', 'subsample']]

,n_estimators,max_depth,learning_rate,colsample_bynode,colsample_bytree,subsample
0,948,12,0.006897,0.78998,0.471958,0.359216


## Feature Importances
With hypertuning we were able to get about a 3 to 7 percent increase in accuracy. The model is still not preforming optimally, but we were able to rule out that the model was not the only issue. The issue might be attributed to the features we used to train. Initially we threw all the features at the model, but that may not be the best option. Luckily, an interesting option for tree models is that they provide feature importances. This allows us to see which features are influencing the model the most and which are doing nothing.

We can grab the top five features from our random forest model and sort them from most influential to least influential.

In [111]:
l = list(enumerate(rfr.feature_importances_))
l.sort(key=lambda x: -x[1])
l[:5 ]

[(7, 0.5560598822450813),
 (3, 0.08794821060257854),
 (11, 0.07518801192606869),
 (2, 0.053176975224588016),
 (4, 0.04076222646361437)]

To visulaize this better, we use altair to graph the importances with their respective column names. We find that they are similer in distribution and both rely on cloud data for the majority of the model.

In [112]:
def plot_importance(tree_model, drop=[], title="Feature Importance"):
    drop.extend(['dni_efficiency', 'ghi_efficiency', 'dhi_efficiency',
                 'STATION','DATE', 'latitude','longitude'])

    source = pd.DataFrame({
        'Feature': list(train.drop(columns=drop)),
        'Importance': list(tree_model.feature_importances_)
    })

    bars = alt.Chart(source, height=500, width = 275, title=title).mark_bar().encode(
        y='Feature:N',
        x="Importance:Q"
    )

    text = bars.mark_text(
        align='left',
        baseline='middle',
        dx=3
    ).encode(
        text=alt.Text('Importance:Q', format=',.2r')
    )

    return (bars + text)
    
alt.hconcat(plot_importance(rfr, title="Random Forest"), plot_importance(xg_reg, title="XGBoost"))

alt.HConcatChart(...)

Next we will dive into changing around the features to see what changes effect the final accuracy. First we will remove one of the top preforming features, cloud_cover, and see what results it gives.

Here we created a function that takes in the columns that the model will not use, train the model, and return a R^2 score and graph. We will use the parameters we found through hypertuning.

In [113]:
def train_plot_model(model, title, cols):    
    #get col list
    dropped_train = train.drop(columns=['dni_efficiency', 'ghi_efficiency', 'dhi_efficiency', 'STATION','DATE',
                                            'latitude','longitude'])
    #drop = [dropped_train.columns.get_loc(col) for col in cols]
    
    # remove cols from transformed data
    #svecs = np.delete(X_train, drop, axis=1)
    svecs = X_train.drop(columns=cols)

    model.fit(svecs, y_train)
    
    #np.delete(X_test, drop,axis=1)
    
    predictions = model.predict(X_test.drop(columns=cols))
    print(title + ": \tR^2 Score: " + str(r2_score(y_test.values, predictions)))
    
    return plot_importance(model, cols.copy(), title=title)
    
dropped_columns = ['cloud_cover']
#alt.hconcat(train_plot_model(rfr, 'Random Forest', dropped_columns),
 #          train_plot_model(xg_reg, 'XGBoost', dropped_columns))

With cloud_cover removed, we are able to see that not much changed in the model. Both models are preforming at the same accuracy and are roughly the same as the preformance prior with cloud_cover included. From this we can conclude that cloud_cover was highly correlated with a combintion of the other columns.

Let's do the opposite of what we tested and remove all the cloud observation features.

In [114]:
dropped_columns = ['overcast', 'cloudy', 'mostly_cloudy',
                   'partly_cloudy', 'mostly_clear', 'clear']
#alt.hconcat(train_plot_model(rfr, 'Random Forest', dropped_columns),
#          train_plot_model(xg_reg, 'XGBoost', dropped_columns))

The model with the cloud_cover feature preformed slightly worse than the cloud observations feature model, so it is safe to say that we can drop cloud_cover as a feature in favor of a better preforming set of features. On the other hand, having a less complex (less features) model could be favorable in some situations as long as the accuracy is not reduced too much. In our case we will opt for the more accurate set of features.

Finally we can try trimming the feature set by removing features that have less than a .02% impact. This will give us the same advantage of having a less complex model but hopefully with less of a drop in accuracy.

In [115]:
dropped_columns = ['station_pressure',
       'wind_speed', 'cloud_cover', 'overcast', 'fog', 'snow',
       'snow_heavy', 'freezing_rain', 'freezing_drizzle', 'ice_pellets',
       'ice_pellets_light', 'ice_pellets_heavy', 'flurries',
       'freezing_rain_heavy', 'freezing_rain_light', 'fog_light']
#alt.hconcat(train_plot_model(rfr, 'Random Forest', dropped_columns),
#          train_plot_model(xg_reg, 'XGBoost', dropped_columns))

In this test, there was an accuracy drop but that is expected when a good number of features are lost. We could continue to fine tune the features we use along with more hyperparameter tuning as well. But from the tests we have done, it looks like there is not much we can do to substantially increase the model's accuracy.

From here we save the original boosting model so that we can use it outside of this notebook. Becouse of the random hypertuning we did, each iteration of this notebook will return a different model that will preform slightly differently. In the next notebook we will look at where the error in this model is coming from, and how to use that information to increase our model's accuracy.

In [116]:
import cloudpickle as cp
import os

def serialize_to(obj, default_filename):
    filename = os.getenv("S2I_PIPELINE_STAGE_SAVE_FILE", default_filename)
    if filename != default_filename:
        cp.dump(obj, open(default_filename, "wb"))
    cp.dump(obj, open(filename, "wb"))

serialize_to(saved_model, "trained_model.sav")